# Importation des données :

In [1]:
pip install openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 5.9 MB/s eta 0:00:0000:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import pandas as pd
import requests
import numpy as np 
from scipy import stats

In [3]:
OECD = pd.read_excel('données/OECD - short term economic indicators.xlsx')

In [4]:
IMF = pd.read_excel('données/IMF - monetary policy.xlsx')

In [5]:
agregated_data=pd.ExcelFile('données/FRBNY-SCE-Data.xlsx')
agregated_anticipation= df1=pd.read_excel(agregated_data, 'Inflation expectations')

In [6]:
agrant = agregated_anticipation[['Unnamed: 1','Unnamed: 2']].drop([0,1,2]).reset_index().rename(columns={'index':'date','Unnamed: 1':'short exp','Unnamed: 2':'mid exp'})

In [7]:
Enq13 = pd.read_excel('https://www.newyorkfed.org/medialibrary/interactives/sce/sce/downloads/data/frbny-sce-public-microdata-complete-13-16.xlsx')

In [8]:
Enq17 = pd.read_excel('https://www.newyorkfed.org/medialibrary/interactives/sce/sce/downloads/data/frbny-sce-public-microdata-complete-17-19.xlsx')

In [9]:
Latest=pd.ExcelFile('https://www.newyorkfed.org/medialibrary/Interactives/sce/sce/downloads/data/frbny-sce-public-microdata-latest.xlsx')
Enq20=pd.read_excel(Latest, 'Data')

In [10]:
Enq20

,"Source: Survey of Consumer Expectations, © 2013-22 Federal Reserve Bank of New York (FRBNY). The SCE data are available without charge at www.newyorkfed.org and may be used subject to license terms posted there. FRBNY disclaims any responsibility or legal liability for this analysis and interpretation of Survey of Consumer Expectations data.",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 210,Unnamed: 211,Unnamed: 212,Unnamed: 213,Unnamed: 214,Unnamed: 215,Unnamed: 216,Unnamed: 217,Unnamed: 218,Unnamed: 219
0,date,userid,tenure,weight,Q1,Q2,Q3,Q4new,Q5new,Q6new,...,Q48,QRA1,QRA2,survey_date,_AGE_CAT,_NUM_CAT,_REGION_CAT,_COMMUTING_ZONE,_EDU_CAT,_HH_INC_CAT
1,202001,70103231,12,0.81825,4,3,20,62,26,44,...,5,NaN,NaN,2020-01-23 00:00:00,40 to 60,High,West,218,College,Over 100k
2,202001,70103252,11,2.879998,3,2,1,30,1,10,...,2,NaN,NaN,2020-01-02 00:00:00,40 to 60,Low,Northeast,203,High School,Under 50k
3,202001,70103258,12,0.549674,4,4,0,3,4,18,...,4,NaN,NaN,2020-01-07 00:00:00,Over 60,High,South,102,College,50k to 100k
4,202001,70103285,12,0.481746,4,4,10,10,10,10,...,1,NaN,NaN,2020-01-27 00:00:00,Under 40,High,Midwest,100,College,50k to 100k
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36351,202205,75006585,1,0.984551,3,3,0,30,50,40,...,4,4,2,2022-05-27 00:00:00,40 to 60,High,West,109,Some College,Over 100k
36352,202205,75006586,1,3.769394,2,1,20,70,50,40,...,4,1,2,2022-05-31 00:00:00,Over 60,High,South,200,High School,Under 50k
36353,202205,75006597,1,0.492543,1,3,20,40,9,26,...,2,3,3,2022-05-19 00:00:00,Under 40,High,Northeast,184,College,Under 50k
36354,202205,75006620,1,0.502195,4,4,0,30,15,30,...,4,4,2,2022-05-24 00:00:00,Under 40,High,West,5,College,Over 100k


In [11]:
#Il faut faire 2-3 manips pour rendre les bases utilisables
new_columns = Enq13.iloc[0]
Enq13.columns = new_columns
Enq13 = Enq13.drop(0)

new_columns = Enq17.iloc[0]
Enq17.columns = new_columns
Enq17 = Enq17.drop(0)

In [12]:
new_columns = Enq20.iloc[0]
Enq20.columns = new_columns
Enq20 = Enq20.drop(0)

In [13]:
Enq17['date'] = pd.to_datetime(Enq17['date'].astype(str) + '01', format='%Y%m%d')
Enq13['date'] = pd.to_datetime(Enq13['date'].astype(str) + '01', format='%Y%m%d')
Enq20['date'] = pd.to_datetime(Enq20['date'].astype(str) + '01', format='%Y%m%d')

In [14]:
Base_finale=pd.concat([Enq13,Enq17,Enq20])

In [15]:
#Base_finale.to_excel('Base_finale.xlsx')

# I-Première spécification :

In [16]:
from sklearn.linear_model import LinearRegression

In [17]:
from statsmodels.regression.linear_model import OLS
import statsmodels.api as sm

In [18]:
AA=Base_finale

In [19]:
list(AA.columns)

['date',
 'userid',
 'tenure',
 'weight',
 'Q1',
 'Q2',
 'Q3',
 'Q4new',
 'Q5new',
 'Q6new',
 'Q8v2',
 'Q8v2part2',
 'Q9_cent25',
 'Q9_cent50',
 'Q9_cent75',
 'Q9_var',
 'Q9_mean',
 'Q9_iqr',
 'Q9_probdeflation',
 'Q9_bin1',
 'Q9_bin2',
 'Q9_bin3',
 'Q9_bin4',
 'Q9_bin5',
 'Q9_bin6',
 'Q9_bin7',
 'Q9_bin8',
 'Q9_bin9',
 'Q9_bin10',
 'Q9bv2',
 'Q9bv2part2',
 'Q9c_cent25',
 'Q9c_cent50',
 'Q9c_cent75',
 'Q9c_var',
 'Q9c_mean',
 'Q9c_iqr',
 'Q9c_probdeflation',
 'Q9c_bin1',
 'Q9c_bin2',
 'Q9c_bin3',
 'Q9c_bin4',
 'Q9c_bin5',
 'Q9c_bin6',
 'Q9c_bin7',
 'Q9c_bin8',
 'Q9c_bin9',
 'Q9c_bin10',
 'Q10_1',
 'Q10_2',
 'Q10_3',
 'Q10_4',
 'Q10_5',
 'Q10_6',
 'Q10_7',
 'Q10_8',
 'Q10_9',
 'Q10_10',
 'Q11',
 'Q12new',
 'ES1_1',
 'ES1_2',
 'ES1_3',
 'ES1_4',
 'ES2',
 'ES3new',
 'ES4',
 'ES5',
 'Q13new',
 'Q14new',
 'Q15',
 'Q16',
 'Q17new',
 'Q18new',
 'Q19',
 'Q20new',
 'Q21new',
 'Q22new',
 'Q23v2',
 'Q23v2part2',
 'Q24_cent25',
 'Q24_cent50',
 'Q24_cent75',
 'Q24_var',
 'Q24_mean',
 'Q24_iqr',
 'Q

In [20]:
educ=pd.get_dummies(AA['_EDU_CAT'])

In [21]:
age=pd.get_dummies(AA['_AGE_CAT'])

In [22]:
income=pd.get_dummies(AA['_HH_INC_CAT'])

In [23]:
numeracy=pd.get_dummies(AA['_NUM_CAT'])

In [24]:
np.sum(educ['High School'])

16124

In [25]:
BC = pd.concat([AA, educ], axis=1).reindex(AA.index) #BC est pour Base complète. Ici, on complète la base avec des dummies

In [26]:
BC = pd.concat([BC, age], axis=1).reindex(BC.index)

In [27]:
BC = pd.concat([BC, income], axis=1).reindex(BC.index)

In [28]:
BC = pd.concat([BC, numeracy], axis=1).reindex(BC.index)

On code maintenant le genre :

In [29]:
%%capture
gender=pd.get_dummies(AA['Q33'])

In [30]:
gender.rename(columns={1: "fem"}, inplace=True)

In [31]:
BC = pd.concat([BC, gender['fem']], axis=1).reindex(BC.index)

C'est fait pour le genre. 

Faisons le ménage dans les non-réponses et dans les outliers :

In [32]:
BC=BC.dropna(subset=['Q8v2part2','Q9bv2part2']) #On supprime les personnes qui n'ont pas répondu aux questions sur les anticipations d'inflation. 

Supprimons les outliers

In [33]:
test=BC[BC['Q8v2part2']<=20]
test=test[test['Q8v2part2']>=-20]

In [34]:
max(test['Q8v2part2'])

20

In [35]:
model = OLS(BC['Q8v2part2'].astype('float'),BC['fem'].astype('float'))
results = model.fit()
su = results.summary()
su

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:              Q8v2part2   R-squared (uncentered):                   0.003
Model:                            OLS   Adj. R-squared (uncentered):              0.003
Method:                 Least Squares   F-statistic:                              407.4
Date:                Mon, 13 Mar 2023   Prob (F-statistic):                    1.86e-90
Time:                        16:04:33   Log-Likelihood:                     -7.4629e+05
No. Observations:              139667   AIC:                                  1.493e+06
Df Residuals:                  139666   BIC:                                  1.493e+06
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
fem           10.5545      0.523     20.183      0.000       9.530      11.579
==============================================================================
Omnibus:                   674137.117   Durbin-Watson:                   1.390
Prob(Omnibus):                  0.000   Jarque-Bera (JB):    9172836698756.129
Skew:                         191.146   Prob(JB):                         0.00
Kurtosis:                   39702.992   Cond. No.                         1.00
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Calcul de l'inflation :

In [36]:
inflation_US = OECD[(OECD['Country'] == "United States") & (OECD['Subject2'] == "Consumer prices: all items")]
inflation_US2 = pd.DataFrame({
    'day': pd.to_datetime(inflation_US['Time'], format='%Y-%m-%d'),
    'Inflation': inflation_US['Value'].astype(float)
})

inflation_US3 = pd.DataFrame({
    'date': inflation_US2['day'],
    'tx_evol_ann_pct': (inflation_US2['Inflation'] / inflation_US2['Inflation'].shift(12) - 1) * 100
})


In [37]:
test['inflation t']=0

In [38]:
df3 = pd.merge(test,inflation_US3,how='outer',left_on=['date'],right_on=['date'])


In [39]:
df3=df3[df3['Q1'].notnull()]

In [40]:
BF=df3

## Définition d'un indicateur d'instabilité personelle :

On va définir les variables qu'il serait pertinent d'inclure : 
- Q3 : proba changement de résidence dans les douze prochain mois
- Q10 : 3 : chomeurs
- Q13new: proba de changer d'emploi
- Q25v2 : 3 : anticipe baisse de revenus
- Q30new : proba de ne pas réussir à payer un prêt



In [41]:
BF.loc[BF['Q25v2']==3, 'loss expectation'] = 1 #On binarise le fait d'anticiper une baisse de ses revenus

In [42]:
BF['indic_perso']=(BF.fillna(0)['Q3']/100+BF.fillna(0)['Q10_3']+BF.fillna(0)['Q13new']/100+BF.fillna(0)['loss expectation']+BF.fillna(0)['Q30new']/100)/5

*j'ai remplacé le 4 de la division par 5 pcq je crois qu'il y a 5 éléments, mais si c'est pas bien supprime ; dans tous les cas ça changera pas grand-chose à la régression puisque c'est proportionnel*

*par contre c'est pas un peu chaud de remplacer tous les nan par des 0?*

Faisons quelques statistiques descriptives dessus :

In [43]:
import statistics as stat

In [44]:
stat.median(BF['indic_perso'])

0.05

In [45]:
stat.mean(BF['indic_perso'])

0.10450771219933926

In [46]:
def weighted_average(dataframe, value, weight):
    val = dataframe[value]
    wt = dataframe[weight]
    return (val * wt).sum() / wt.sum()

In [47]:
weighted_average(BF, 'indic_perso', 'weight')

0.10622211144606332

In [48]:
stat.pstdev(BF['indic_perso'])

0.12193138591413451

In [49]:
len(BF['Q8v2part2'])

129548

In [122]:
BF.dropna(subset=['weight'], inplace=True)

In [123]:
poids = BF['weight'].astype('float')

In [124]:
Y = BF['Q8v2part2'].astype('float')
X = BF.fillna(0)[['tx_evol_ann_pct','indic_perso']]
X = sm.add_constant(X)
model = sm.OLS(Y,X)
results = model.fit()
su = results.summary()
su

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              Q8v2part2   R-squared:                       0.025
Model:                            OLS   Adj. R-squared:                  0.025
Method:                 Least Squares   F-statistic:                     1690.
Date:                Mon, 13 Mar 2023   Prob (F-statistic):               0.00
Time:                        17:07:40   Log-Likelihood:            -3.9566e+05
No. Observations:              129518   AIC:                         7.913e+05
Df Residuals:                  129515   BIC:                         7.914e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               2.9957      0.025    118.143      0.000       2.946       3.045
tx_evol_ann_pct     0.4309      0.008     55.583      0.000       0.416       0.446
indic_perso         2.2426      0.117     19.153      0.000       2.013       2.472
==============================================================================
Omnibus:                    13297.398   Durbin-Watson:                   1.977
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           110409.357
Skew:                           0.098   Prob(JB):                         0.00
Kurtosis:                       7.519   Cond. No.                         24.2
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Alternative : on sépare les différents aspects de l'instabilité personnelle pour ne pas leur donner un poids arbitraire

In [51]:
#Allure proba de changement de résidence sous 12 mois
BF['Q3']

0         50
1         25
2          3
3          0
4         10
          ..
129590     0
129591     0
129592    20
129593     0
129594    20
Name: Q3, Length: 129548, dtype: object

In [52]:
print("nb de valeurs manquantes dans Q3 : "+str(BF['Q3'].isnull().sum()))

nb de valeurs manquantes dans Q3 : 440


In [53]:
#Allure de l'indicatrice du chômage
BF['Q10_3']

0         0
1         0
2         0
3         0
4         0
         ..
129590    0
129591    0
129592    0
129593    0
129594    0
Name: Q10_3, Length: 129548, dtype: object

In [54]:
print("nb de valeurs manquantes dans Q10_3 : "+str(BF['Q10_3'].isnull().sum()))

nb de valeurs manquantes dans Q10_3 : 0


le chômage est tjrs renseigné, c'est cool

In [55]:
#Allure proba changer d'emploi
BF['Q13new']

0         NaN
1         NaN
2         NaN
3         NaN
4          30
         ... 
129590      0
129591      0
129592    NaN
129593     75
129594     15
Name: Q13new, Length: 129548, dtype: object

In [56]:
print("nb de valeurs manquantes dans Q13new : "+str(BF['Q13new'].isnull().sum()))

nb de valeurs manquantes dans Q13new : 53054


*c'est une catastrophe absolue, quasiment la moitié des participants n'y ont pas répondu...
il faudra sans doute faire une régression qui ne le prend pas en compte*

In [57]:
#Allure anticipation baisse revenus
BF['Q25v2']

0         3
1         1
2         1
3         1
4         3
         ..
129590    1
129591    1
129592    3
129593    1
129594    1
Name: Q25v2, Length: 129548, dtype: object

In [58]:
print("nb de valeurs manquantes dans Q25v2 : "+str(BF['Q25v2'].isnull().sum()))

nb de valeurs manquantes dans Q25v2 : 60


In [59]:
#Allure proba ne pas réussir à rembourser un prêt
BF['Q30new']

0         30
1          0
2         97
3          0
4          5
          ..
129590     0
129591     0
129592     0
129593     0
129594     5
Name: Q30new, Length: 129548, dtype: object

In [60]:
print("nb de valeurs manquantes dans Q30new : "+str(BF['Q30new'].isnull().sum()))

nb de valeurs manquantes dans Q30new : 226


In [61]:
BF

,date,userid,tenure,weight,Q1,Q2,Q3,Q4new,Q5new,Q6new,...,50k to 100k,Over 100k,Under 50k,High,Low,fem,inflation t,tx_evol_ann_pct,loss expectation,indic_perso
0,2013-06-01,70000220,6,16.327999,3,3,50,20,10,15,...,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.754417,1.0,0.36
1,2013-06-01,70000224,7,0.228,4,4,25,10,25,75,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.754417,NaN,0.05
2,2013-06-01,70000234,6,4.066,4,3,3,9,20,20,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.754417,NaN,0.20
3,2013-06-01,70000238,6,3.035,3,3,0,10,5,70,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.754417,NaN,0.00
4,2013-06-01,70000239,7,1.947,2,3,10,20,20,50,...,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.754417,1.0,0.29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129590,2022-05-01,75006584,1,0.964475,3,2,0,35,15,23,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,8.581512,NaN,0.00
129591,2022-05-01,75006585,1,0.984551,3,3,0,30,50,40,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,8.581512,NaN,0.00
129592,2022-05-01,75006586,1,3.769394,2,1,20,70,50,40,...,0.0,0.0,1.0,1.0,0.0,1.0,0.0,8.581512,1.0,0.24
129593,2022-05-01,75006620,1,0.502195,4,4,0,30,15,30,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,8.581512,NaN,0.15


In [62]:
instabilité = ['Q3', 'Q10_3', 'Q13new', 'Q25v2', 'Q30new']

In [63]:
#on fabrique un dataframe sans les valeurs manquantes
BF2 = BF.dropna(subset=instabilité)

In [64]:
%%capture
#du coup on re binarise
BF2.loc[BF['Q25v2']==3, 'loss expectation'] = 1
BF2['loss expectation']=BF2.fillna(0)['loss expectation']

In [65]:
instabilité = ['Q3', 'Q10_3', 'Q13new', 'loss expectation', 'Q30new']

In [66]:
%%capture
#transformation des colonnes en float
for element in instabilité :
    BF2[element] = BF2[element].astype(float)

In [67]:
BF2 = BF2.rename(columns={'Q3': 'proba chgt résidence',
                          'Q10_3': 'chômage',
                          'Q13new': 'proba chgt emploi',
                          'loss expectation': 'proba perte revenus',
                          'Q30new': 'proba ne pas réussir à rembourser un prêt'})

In [119]:
BF2.dropna(subset=['weight'], inplace=True)

In [120]:
instabilité = ['proba chgt résidence',
               'chômage',
               'proba chgt emploi',
               'proba perte revenus',
               'proba ne pas réussir à rembourser un prêt']
poids = BF2['weight'].astype('float')

In [121]:
Y = BF2['Q8v2part2'].astype('float')
X = BF2[instabilité]
X = sm.add_constant(X)
model = sm.WLS(Y,X, weights=poids)
results = model.fit()
su = results.summary()
su

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:              Q8v2part2   R-squared:                       0.004
Model:                            WLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                     64.27
Date:                Mon, 13 Mar 2023   Prob (F-statistic):           3.58e-67
Time:                        17:06:00   Log-Likelihood:            -2.4572e+05
No. Observations:               76312   AIC:                         4.915e+05
Df Residuals:                   76306   BIC:                         4.915e+05
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
=============================================================================================================
                                                coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------
const                                         4.0044      0.028    142.278      0.000       3.949       4.060
proba chgt résidence                         -0.0041      0.001     -5.917      0.000      -0.005      -0.003
chômage                                      -1.1027      0.718     -1.537      0.124      -2.509       0.304
proba chgt emploi                             0.0050      0.001      4.630      0.000       0.003       0.007
proba perte revenus                           0.2378      0.071      3.348      0.001       0.099       0.377
proba ne pas réussir à rembourser un prêt     0.0146      0.001     14.807      0.000       0.013       0.017
==============================================================================
Omnibus:                    14850.756   Durbin-Watson:                   1.935
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           431424.549
Skew:                           0.192   Prob(JB):                         0.00
Kurtosis:                      14.642   Cond. No.                     1.43e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.43e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Les coeff sont tous significatifs à 95%, mais très petits

## Maintenant, essayons de définir un indicateur pour l'instabilité macro :

On va noter ici les variables qui pourraient être utiles pour définir un tel indicateur :
- Q4new: probabilité d'augmentation du chômage dans les douze prochain mois
- Q5new: proba d'une augmentation des taux d'intêret
- Q6new: proba d'une augmentation des indices boursiers: 100-Q6new = chance d'une baisse donc


In [70]:
BF['indic_macro']=(BF.fillna(0)['Q4new']/100+BF.fillna(0)['Q5new']/100+(100-BF.fillna(0)['Q6new'])/100)/3

Maintenant, faisons quelques statistiques descriptives :

In [71]:
stat.median(BF['indic_macro'])

0.4133333333333334

In [72]:
stat.mean(BF['indic_macro'])

0.4222305631889338

In [73]:
weighted_average(BF, 'indic_macro', 'weight')

0.41974160136324934

In [74]:
stat.pstdev(BF['indic_macro'])

0.12307821901643654

In [75]:
BF['indic_macro']

0         0.383333
1         0.200000
2         0.363333
3         0.150000
4         0.300000
            ...   
129590    0.423333
129591    0.466667
129592    0.600000
129593    0.383333
129594    0.503333
Name: indic_macro, Length: 129548, dtype: float64

In [76]:
Y = BF['Q8v2part2'].astype('float')
X = BF.fillna(0)[['tx_evol_ann_pct','indic_perso','indic_macro']]
X = sm.add_constant(X)
model = sm.OLS(Y,X)
results = model.fit()
su = results.summary()
su

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              Q8v2part2   R-squared:                       0.029
Model:                            OLS   Adj. R-squared:                  0.029
Method:                 Least Squares   F-statistic:                     1277.
Date:                Mon, 13 Mar 2023   Prob (F-statistic):               0.00
Time:                        16:06:34   Log-Likelihood:            -3.9555e+05
No. Observations:              129548   AIC:                         7.911e+05
Df Residuals:                  129544   BIC:                         7.911e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               2.0034      0.054     37.377      0.000       1.898       2.108
tx_evol_ann_pct     0.4276      0.008     55.224      0.000       0.412       0.443
indic_perso         1.8780      0.118     15.894      0.000       1.646       2.110
indic_macro         2.4568      0.117     21.000      0.000       2.228       2.686
==============================================================================
Omnibus:                    13314.131   Durbin-Watson:                   1.976
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           110808.158
Skew:                           0.097   Prob(JB):                         0.00
Kurtosis:                       7.527   Cond. No.                         26.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Idem, alternative sans bloc :

In [77]:
BF['Q4new']

0         20
1         10
2          9
3         10
4         20
          ..
129590    35
129591    30
129592    70
129593    30
129594    80
Name: Q4new, Length: 129548, dtype: object

In [78]:
print("nb de valeurs manquantes dans Q4new : "+str(BF['Q4new'].isnull().sum()))

nb de valeurs manquantes dans Q4new : 255


In [79]:
BF['Q5new']

0         10
1         25
2         20
3          5
4         20
          ..
129590    15
129591    50
129592    50
129593    15
129594    51
Name: Q5new, Length: 129548, dtype: object

In [80]:
print("nb de valeurs manquantes dans Q5new : "+str(BF['Q5new'].isnull().sum()))

nb de valeurs manquantes dans Q5new : 241


In [81]:
BF['Q6new']

0         15
1         75
2         20
3         70
4         50
          ..
129590    23
129591    40
129592    40
129593    30
129594    80
Name: Q6new, Length: 129548, dtype: object

In [82]:
print("nb de valeurs manquantes dans Q6new : "+str(BF['Q6new'].isnull().sum()))

nb de valeurs manquantes dans Q6new : 810


Au total on va perdre pas mal de répondants, mais ça reste pas énorme

Se posera tout de même la question, si l'on envisage une grosse régression, du nombre d'individus qui auront répondu à tout ce qui nous intéresse ; d'ailleurs étudions ça avant de passer à la suite

In [83]:
len(BF[BF['Q3'].isna() | 
       BF['Q10_3'].isna()  |
       BF['Q13new'].isna() | 
       BF['Q25v2'].isna() | 
       BF['Q30new'].isna() | 
       BF['Q4new'].isna() | 
       BF['Q5new'].isna() | 
       BF['Q6new'].isna()])

53587

On perd la moitié des répondants. Tandis que sans la question de la proba de changer d'emploi :

In [84]:
len(BF[BF['Q3'].isna() |
       BF['Q10_3'].isna() |
       BF['Q25v2'].isna() |
       BF['Q30new'].isna() |
       BF['Q4new'].isna() | 
       BF['Q5new'].isna() |
       BF['Q6new'].isna()])

1179

Ce qui est beaucoup plus raisonnable. On l'exclura donc sans doute.

revenons à nos moutons

In [85]:
instabilité2 = ['Q4new', 'Q5new', 'Q6new']

In [86]:
#on fabrique un dataframe sans les valeurs manquantes
BF3 = BF.dropna(subset=instabilité2)

In [87]:
%%capture
#transformation des colonnes en float
for element in instabilité2 :
    BF3[element] = BF3[element].astype(float)

In [88]:
%%capture
#obtention d'une colonne proba diminution indices boursiers
BF3['Q6new_bis'] = 100-BF3['Q6new']

In [89]:
BF3 = BF3.rename(columns={'Q4new': 'proba augmentation chô sous douze mois',
                          'Q5new': 'proba augmentation tx intêret',
                          'Q6new_bis' : 'proba diminution indices boursiers'})

In [114]:
BF3.dropna(subset=['weight'], inplace=True)

In [115]:
instabilité2 = ['proba augmentation chô sous douze mois',
                'proba augmentation tx intêret',
                'proba diminution indices boursiers']
poids = BF3['weight'].astype('float')

In [116]:
Y = BF3['Q8v2part2'].astype('float')
X = BF3[instabilité2]
X = sm.add_constant(X)
model = sm.WLS(Y,X, weights=poids)
results = model.fit()
su = results.summary()
su

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:              Q8v2part2   R-squared:                       0.028
Model:                            WLS   Adj. R-squared:                  0.028
Method:                 Least Squares   F-statistic:                     1220.
Date:                Mon, 13 Mar 2023   Prob (F-statistic):               0.00
Time:                        17:03:54   Log-Likelihood:            -4.1988e+05
No. Observations:              128627   AIC:                         8.398e+05
Df Residuals:                  128623   BIC:                         8.398e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==========================================================================================================
                                             coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------
const                                      2.4086      0.066     36.682      0.000       2.280       2.537
proba augmentation chô sous douze mois     0.0320      0.001     45.158      0.000       0.031       0.033
proba augmentation tx intêret             -0.0151      0.001    -21.249      0.000      -0.017      -0.014
proba diminution indices boursiers         0.0237      0.001     30.953      0.000       0.022       0.025
==============================================================================
Omnibus:                    23647.999   Durbin-Watson:                   1.941
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           660379.465
Skew:                           0.019   Prob(JB):                         0.00
Kurtosis:                      14.100   Cond. No.                         321.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Tous significatifs à 95%, mais coefficients ridicules

## Tentative de maxi régression

On va inclure tout sans bloc ; sans considérer la question qui fait perdre la moitié des répondants

In [92]:
instabilité_micro = ['Q3', 'Q10_3', 'Q25v2', 'Q30new']
instabilité_macro = ['Q4new', 'Q5new', 'Q6new']
régresseurs = instabilité_micro + instabilité_macro + ['tx_evol_ann_pct', 'fem']

In [93]:
#retrait des valeurs manquantes
BF4 = BF.dropna(subset=régresseurs)

In [94]:
%%capture
#du coup on re binarise Q25v2
BF4.loc[BF['Q25v2']==3, 'loss expectation'] = 1
BF4['loss expectation']=BF4.fillna(0)['loss expectation']

In [95]:
%%capture
#obtention d'une colonne proba diminution indices boursiers
BF4['Q6new_bis'] = 100-BF4['Q6new']
BF4['Q6new_bis'] = BF4['Q6new_bis'].astype(float)

In [96]:
%%capture
#transformation des colonnes en float
for element in régresseurs :
    BF4[element] = BF4[element].astype(float)

In [97]:
BF4 = BF4.rename(columns={'Q4new': 'proba augmentation chô sous douze mois', 
                          'Q5new': 'proba augmentation tx intêret',
                          'Q6new_bis' : 'proba diminution indices boursiers',
                          'Q3': 'proba chgt résidence',
                          'Q10_3': 'chômage',
                          'loss expectation': 'proba perte revenus',
                          'Q30new': 'proba ne pas réussir à rembourser un prêt'})

In [112]:
BF4.dropna(subset=['weight'], inplace=True)

In [111]:
régresseurs = ['proba augmentation chô sous douze mois',
               'proba augmentation tx intêret',
               'proba diminution indices boursiers',
               'proba chgt résidence',
               'chômage',
               'proba perte revenus',
               'proba ne pas réussir à rembourser un prêt',
               'fem',
               'tx_evol_ann_pct']
poids = BF4['weight'].astype('float')

In [113]:
Y = BF4['Q8v2part2'].astype('float')
X = BF4[régresseurs]
X = sm.add_constant(X)
model = sm.WLS(Y,X, weights=poids)
results = model.fit()
su = results.summary()
su

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:              Q8v2part2   R-squared:                       0.053
Model:                            WLS   Adj. R-squared:                  0.053
Method:                 Least Squares   F-statistic:                     805.3
Date:                Mon, 13 Mar 2023   Prob (F-statistic):               0.00
Time:                        17:00:13   Log-Likelihood:            -4.1719e+05
No. Observations:              128341   AIC:                         8.344e+05
Df Residuals:                  128331   BIC:                         8.345e+05
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
=============================================================================================================
                                                coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------
const                                         1.4260      0.068     21.051      0.000       1.293       1.559
proba augmentation chô sous douze mois        0.0308      0.001     43.082      0.000       0.029       0.032
proba augmentation tx intêret                -0.0164      0.001    -23.280      0.000      -0.018      -0.015
proba diminution indices boursiers            0.0219      0.001     28.804      0.000       0.020       0.023
proba chgt résidence                         -0.0039      0.001     -6.647      0.000      -0.005      -0.003
chômage                                       0.3246      0.085      3.824      0.000       0.158       0.491
proba perte revenus                           0.2297      0.047      4.866      0.000       0.137       0.322
proba ne pas réussir à rembourser un prêt     0.0147      0.001     19.010      0.000       0.013       0.016
fem                                           1.1836      0.067     17.679      0.000       1.052       1.315
tx_evol_ann_pct                               0.4551      0.009     52.851      0.000       0.438       0.472
==============================================================================
Omnibus:                    24620.283   Durbin-Watson:                   1.980
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           753471.561
Skew:                          -0.092   Prob(JB):                         0.00
Kurtosis:                      14.869   Cond. No.                         437.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

mouais apres on peut voir avec des train/tests

# La deuxième spécification :

In [100]:
list(BF.head())

['date',
 'userid',
 'tenure',
 'weight',
 'Q1',
 'Q2',
 'Q3',
 'Q4new',
 'Q5new',
 'Q6new',
 'Q8v2',
 'Q8v2part2',
 'Q9_cent25',
 'Q9_cent50',
 'Q9_cent75',
 'Q9_var',
 'Q9_mean',
 'Q9_iqr',
 'Q9_probdeflation',
 'Q9_bin1',
 'Q9_bin2',
 'Q9_bin3',
 'Q9_bin4',
 'Q9_bin5',
 'Q9_bin6',
 'Q9_bin7',
 'Q9_bin8',
 'Q9_bin9',
 'Q9_bin10',
 'Q9bv2',
 'Q9bv2part2',
 'Q9c_cent25',
 'Q9c_cent50',
 'Q9c_cent75',
 'Q9c_var',
 'Q9c_mean',
 'Q9c_iqr',
 'Q9c_probdeflation',
 'Q9c_bin1',
 'Q9c_bin2',
 'Q9c_bin3',
 'Q9c_bin4',
 'Q9c_bin5',
 'Q9c_bin6',
 'Q9c_bin7',
 'Q9c_bin8',
 'Q9c_bin9',
 'Q9c_bin10',
 'Q10_1',
 'Q10_2',
 'Q10_3',
 'Q10_4',
 'Q10_5',
 'Q10_6',
 'Q10_7',
 'Q10_8',
 'Q10_9',
 'Q10_10',
 'Q11',
 'Q12new',
 'ES1_1',
 'ES1_2',
 'ES1_3',
 'ES1_4',
 'ES2',
 'ES3new',
 'ES4',
 'ES5',
 'Q13new',
 'Q14new',
 'Q15',
 'Q16',
 'Q17new',
 'Q18new',
 'Q19',
 'Q20new',
 'Q21new',
 'Q22new',
 'Q23v2',
 'Q23v2part2',
 'Q24_cent25',
 'Q24_cent50',
 'Q24_cent75',
 'Q24_var',
 'Q24_mean',
 'Q24_iqr',
 'Q

On doit définir les intervalles qui vont nous intéresser ici :
- pour l'inflation à court terme: Q9 inflation supérieur à 4% (bin 1, bin 2 et bin 3)
- déflation : bin6 à bin 10

- long terme: Q9c+les mêmes bin

N.B : On considère pour la suite que les anticipations non remplies sont de 10 ( équiprobabilit de chaque tranche )

In [ ]:
%%capture
BF['proba_st_high']=(BF['Q9_bin1'].fillna(10)+BF['Q9_bin2'].fillna(10)+BF['Q9_bin3'].fillna(10))

In [102]:
%%capture
BF['proba_st_defl']=(BF['Q9_bin6'].fillna(10)+BF['Q9_bin7'].fillna(10)+BF['Q9_bin8'].fillna(10)+BF['Q9_bin9'].fillna(10)+BF['Q9_bin10'].fillna(10))

In [103]:
%%capture
BF['proba_lt_high']=(BF['Q9c_bin1'].fillna(10)+BF['Q9c_bin2'].fillna(10)+BF['Q9c_bin3'].fillna(10))

In [104]:
%%capture
BF['proba_lt_defl']=(BF['Q9c_bin6'].fillna(10)+BF['Q9c_bin7'].fillna(10)+BF['Q9c_bin8'].fillna(10)+BF['Q9c_bin9'].fillna(10)+BF['Q9c_bin10'].fillna(10))

### A - Short term - high :

In [105]:
%%capture
BF.rename(columns={'High': "High_num"}, inplace=True)

In [125]:
Y = BF['proba_st_high'].astype('float')
X = BF.fillna(0)[['tx_evol_ann_pct','indic_perso','indic_macro','fem','Over 60','Under 40','College','Over 100k','Under 50k','High_num']]
X = sm.add_constant(X)
model = sm.WLS(Y,X, weights=poids)
results = model.fit()
su = results.summary()
su

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:          proba_st_high   R-squared:                       0.092
Model:                            WLS   Adj. R-squared:                  0.092
Method:                 Least Squares   F-statistic:                     1317.
Date:                Mon, 13 Mar 2023   Prob (F-statistic):               0.00
Time:                        17:09:03   Log-Likelihood:            -6.5176e+05
No. Observations:              129518   AIC:                         1.304e+06
Df Residuals:                  129507   BIC:                         1.304e+06
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              19.7609      0.421     46.952      0.000      18.936      20.586
tx_evol_ann_pct     4.1166      0.051     80.648      0.000       4.017       4.217
indic_perso        12.6281      0.780     16.185      0.000      11.099      14.157
indic_macro        28.1805      0.795     35.449      0.000      26.622      29.739
fem                 7.2845      0.393     18.527      0.000       6.514       8.055
Over 60             2.7775      0.224     12.423      0.000       2.339       3.216
Under 40           -4.1733      0.238    -17.555      0.000      -4.639      -3.707
College            -6.3165      0.216    -29.206      0.000      -6.740      -5.893
Over 100k          -3.3801      0.252    -13.427      0.000      -3.873      -2.887
Under 50k           3.2926      0.229     14.399      0.000       2.844       3.741
High_num           -4.0171      0.213    -18.850      0.000      -4.435      -3.599
==============================================================================
Omnibus:                    12229.387   Durbin-Watson:                   1.952
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            17898.526
Skew:                           0.739   Prob(JB):                         0.00
Kurtosis:                       4.064   Cond. No.                         29.0
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### B - Short term - deflation :

In [126]:
Y = BF['proba_st_defl'].astype('float')
X = BF.fillna(0)[['tx_evol_ann_pct','indic_perso','indic_macro','fem','Over 60','Under 40','College','Over 100k','Under 50k','High_num']]
X = sm.add_constant(X)
model = sm.WLS(Y,X, weights = poids)
results = model.fit()
su = results.summary()
su

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:          proba_st_defl   R-squared:                       0.050
Model:                            WLS   Adj. R-squared:                  0.050
Method:                 Least Squares   F-statistic:                     683.5
Date:                Mon, 13 Mar 2023   Prob (F-statistic):               0.00
Time:                        17:09:54   Log-Likelihood:            -6.1208e+05
No. Observations:              129518   AIC:                         1.224e+06
Df Residuals:                  129507   BIC:                         1.224e+06
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              22.5329      0.310     72.735      0.000      21.926      23.140
tx_evol_ann_pct    -0.7513      0.038    -19.995      0.000      -0.825      -0.678
indic_perso         6.2682      0.574     10.914      0.000       5.143       7.394
indic_macro        -4.9386      0.585     -8.440      0.000      -6.085      -3.792
fem                 5.0780      0.289     17.546      0.000       4.511       5.645
Over 60            -1.6484      0.165    -10.017      0.000      -1.971      -1.326
Under 40            2.2887      0.175     13.080      0.000       1.946       2.632
College            -2.7638      0.159    -17.362      0.000      -3.076      -2.452
Over 100k          -0.4144      0.185     -2.236      0.025      -0.778      -0.051
Under 50k           3.3531      0.168     19.921      0.000       3.023       3.683
High_num           -7.3930      0.157    -47.131      0.000      -7.700      -7.086
==============================================================================
Omnibus:                    87198.515   Durbin-Watson:                   1.989
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          2443428.481
Skew:                           2.849   Prob(JB):                         0.00
Kurtosis:                      23.501   Cond. No.                         29.0
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### C-Long terme - high :

In [127]:
Y = BF['proba_lt_high'].astype('float')
X = BF.fillna(0)[['tx_evol_ann_pct','indic_perso','indic_macro','fem','Over 60','Under 40','College','Over 100k','Under 50k','High_num']]
X = sm.add_constant(X)
model = sm.WLS(Y,X, weights = poids)
results = model.fit()
su = results.summary()
su

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:          proba_lt_high   R-squared:                       0.050
Model:                            WLS   Adj. R-squared:                  0.050
Method:                 Least Squares   F-statistic:                     680.2
Date:                Mon, 13 Mar 2023   Prob (F-statistic):               0.00
Time:                        17:10:28   Log-Likelihood:            -6.5430e+05
No. Observations:              129518   AIC:                         1.309e+06
Df Residuals:                  129507   BIC:                         1.309e+06
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              24.7503      0.429     57.666      0.000      23.909      25.592
tx_evol_ann_pct     1.7989      0.052     34.558      0.000       1.697       1.901
indic_perso         9.9063      0.796     12.450      0.000       8.347      11.466
indic_macro        25.8421      0.811     31.876      0.000      24.253      27.431
fem                 6.7737      0.401     16.893      0.000       5.988       7.560
Over 60             3.0352      0.228     13.312      0.000       2.588       3.482
Under 40           -4.1440      0.242    -17.094      0.000      -4.619      -3.669
College            -5.5035      0.221    -24.953      0.000      -5.936      -5.071
Over 100k          -2.7172      0.257    -10.585      0.000      -3.220      -2.214
Under 50k           3.3883      0.233     14.530      0.000       2.931       3.845
High_num           -3.7418      0.217    -17.218      0.000      -4.168      -3.316
==============================================================================
Omnibus:                    10405.144   Durbin-Watson:                   1.971
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            13653.658
Skew:                           0.706   Prob(JB):                         0.00
Kurtosis:                       3.731   Cond. No.                         29.0
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### D - Long term - Deflation :

In [ ]:
Y = BF['proba_lt_defl'].astype('float')
X = BF.fillna(0)[['tx_evol_ann_pct','indic_perso','indic_macro','fem','Over 60','Under 40','College','Over 100k','Under 50k','High_num']]
X = sm.add_constant(X)
model = sm.WLS(Y,X, weights = poids)
results = model.fit()
su = results.summary()
su